In [1]:
from scipy.optimize import fmin_slsqp
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import log
import os

from pulp import *

In [2]:
os.chdir('D:\OneDrive - Georgia State University\Data Science\Personal_Project\Optimization - Loan Repayment')

Reference:

1) [Linear Programming in Python](https://hackernoon.com/linear-programming-in-python-a-straight-forward-tutorial-a0d152618121)

## Compound Interest



In [3]:
def compound_interest(PV, APR, T):
    FV = PV(1+(APR/12)**(12*T))
    return FV

# Plan #1) Minimum Payment in certain period 

## 1-1) Formula

![Minimum_Payment](image/Minimum_Payment.jpg)

In [4]:
def minimum_payment(PV, APR, N):
    i = (APR/12)/100
    PMT = PV * i /(1-(1+i)**(-N))
    
    return PMT

In [5]:
minimum_payment(100, 5, 1)

100.41666666666703

In [6]:
10.184 + 41.108 + 41.108

92.39999999999999

## 1-2) Optimization

In [7]:
def payment_per_month(expected_scores, prices, positions, clubs, difficulties, chance_of_playing_next_round, total_budget=100, sub_factor=0.2, difficulty=2, chance_next = 75):
    num_players = len(expected_scores)
    model = pulp.LpProblem("Constrained value maximization", pulp.LpMaximize)
    
    decisions = [
        pulp.LpVariable("x{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    captain_decisions = [
        pulp.LpVariable("y{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    sub_decisions = [
        pulp.LpVariable("z{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]


    # objective function:
    model += sum((captain_decisions[i] + decisions[i] + (sub_decisions[i]*sub_factor)) * expected_scores[i]
                 for i in range(num_players)), "Objective"

    # cost constraint
    model += sum((decisions[i] + sub_decisions[i]) * prices[i] for i in range(num_players)) <= total_budget  # total cost

    # position constraints and difficulty contraints and chance_of_playing_next_round 
    # 1 starting goalkeeper
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 1 and difficulties[i] <= difficulty and chance_of_playing_next_round[i] > chance_next) == 1
    # 2 total goalkeepers
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 1) == 2

    # 3-5 starting defenders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 2 and difficulties[i] <= difficulty and chance_of_playing_next_round[i] > chance_next) >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 2 and difficulties[i] <= difficulty and chance_of_playing_next_round[i] > chance_next) <= 5
    # 5 total defenders
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 2) == 5

    # 3-5 starting midfielders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 3 and difficulties[i] <= difficulty and chance_of_playing_next_round[i] > chance_next) >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 3 and difficulties[i] <= difficulty and chance_of_playing_next_round[i] > chance_next) <= 5
    # 5 total midfielders
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 3) == 5

    # 1-3 starting attackers
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 4 and difficulties[i] <= difficulty and chance_of_playing_next_round[i] > chance_next) >= 1
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 4 and difficulties[i] <= difficulty and chance_of_playing_next_round[i] > chance_next) <= 3
    # 3 total attackers
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 4) == 3

    # club constraint
    for club_id in np.unique(clubs):
        model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if clubs[i] == club_id) <= 3  # max 3 players

    model += sum(decisions) == 11  # total team size
    model += sum(captain_decisions) == 1  # 1 captain
    
    for i in range(num_players):  
        model += (decisions[i] - captain_decisions[i]) >= 0  # captain must also be on team
        model += (decisions[i] + sub_decisions[i]) <= 1  # subs must not be on team

    
    
    model.solve()
    status = model.solve()
    print(f'This problem is {LpStatus[status]}')

    print("Total expected score = {}".format(model.objective.value()))

    return decisions, captain_decisions, sub_decisions


## 1-3) Calculating the outstanding balance

![Remaining_Balance](image/Remaining_Balance.jpg)

In [8]:
def remaining_balance(PV, PMT, APR, n):
    
    i = (APR/12)/100
    B = (PV*(1+i)**n) - ((PMT*((1+i)**(n)-1)/i))
    
    return B

In [9]:
remaining_balance(300, 10, 6, 10)

213.06177553065768

## Constraints

## 1-4) How long does it take to pay off with P amount

![How_long_before_the_debt_payoff](image/How_long_before_the_debt_payoff.jpg)

In [10]:
'''payment must exceeds the accrued interest each month
accrued interest = PV * (1+i) '''

def how_long(PV, APR, PMT):
    i = (APR/12)/100
    accrued_interest = PV * i
    try:
        N = round(-(log(1-((PV*i)/PMT)))/log(1+i))
        return N
    
    except:
        print(f"Your payment must be greater than accrued_interest which is {'%.2f' % accrued_interest}.")
    


In [11]:
how_long(100, 6, 10)

Your payment must be greater than accrued_interest which is 0.50.


# 2) Optimizing for case #2 which optimizes the distribution of payment

In [12]:
My_Loans = {
    'Name' : ['BOA', 'Sapire','Southwest','SallieMae'],
    'Principal' : [350.59, 1672.04, 597.32, 766.63],
    'APR' : [6, 4, 4, 5]
}
My_Loans = pd.DataFrame(My_Loans)


In [14]:
def how_long(PV, APR, Max_PMT):
    num_loans = len(PV)
    model = pulp.LpProblem("Repayment Period Minimization", pulp.LpMinimize)
    
    repayment_amount = [
        pulp.LpVariable("x{}".format(i), lowBound=0, cat='Continuous')
        for i in range(num_loans)
    ]
    

    # objective function:
    model += sum(((-(log(1-((PV[i]*((APR[i]/12)/100))/(repayment_amount[i]))))/log(1+((APR[i]/12)/100)))) 
                 for i in range(num_loans)), "Objective"

    # greater than accrued interest constraint
    model += (repayment_amount[i] for i in range(num_loans))  > (PV[i]*(APR[i]/12)/100)

    # less than the total repayment amount
    model += sum(repayment_amount[i] for i in range(num_loans)) <= Max_PMT
    
    
    model.solve()
    status = model.solve()
    print(f'This problem is {LpStatus[status]}')

    print("Total time takes = {}".format(model.objective.value()))

    return repayment_amount


In [38]:
# Constants
# Create a list of the food items
loan_items = My_Loans['Name']
# Principals
PV = My_Loans['Principal']

# APR
APR = My_Loans['APR']


# Maximum PMT ----- This will be an input
Max_PMT = 2000

period = how_long(PV, APR, Max_PMT)

TypeError: unsupported operand type(s) for /: 'float' and 'LpVariable'

In [ ]:
it = (APR[i]/12)/100
round(-(log(1-((PV*it)/PMT)))/log(1+it))

'''payment must exceeds the accrued interest each month
accrued interest = PV * (1+i) '''

def how_long(PV, APR, PMT):
    i = (APR/12)/100
    accrued_interest = PV * i
    try:
        N = round(-(log(1-((PV*i)/PMT)))/log(1+i))
        return N
    
    except:
        print(f"Your payment must be greater than accrued_interest which is {'%.2f' % accrued_interest}.")
    


In [14]:
# Decision Variable --- PMT

num_loans = len(My_Loans) 
loan_items
PMT = LpVariable.dicts("PMT", list(range(num_loans)), 0, Max_PMT, cat="Continuous")

In [50]:
# Goal Constraint
c1 = sum(PMT) == Max_PMT

# adding the constraints to the problem
problem += c1

In [ ]:
# objective function
problem += -factory_days[0]*cf0*f0 - factory_days[1]*cf1*f1 - factory_days[2]*cf2*f2

In [15]:
# Objective Function
empty = []

for j in range(len(My_Loans)):
    i = (APR[j]/12)/100
    T = round(-(log(1-((PV[j]*i)/PMT[j])))/log(1+i))


Your payment must be greater than accrued_interest which is 1.75.
Your payment must be greater than accrued_interest which is 5.57.
Your payment must be greater than accrued_interest which is 1.99.
Your payment must be greater than accrued_interest which is 3.19.


In [15]:
PMT = [0] * len(My_Loans)
PMT

[0, 0, 0, 0]

Min(T) = how_long(PV1, APR1, PMT1) + how_long(PV2, APR2, PMT2) + how_long(PV3, APR3, PMT3)

S.T.

sum of PMTi > 0
sum of PMTi < Affordable Payment
PTMi < PVi

In [17]:
# Constants

# Principals

350.59